Lecture 4: SQL
======================

Set Operators
=======
We generate three tables below:
* R is {1,2,3,4,5}
* S is {}
* T is {1,4,7,10}


In [8]:
# Create tables & insert some random numbers
# Note: in Postgresql, try the generate_series function...
%sql DROP TABLE IF EXISTS R; DROP TABLE IF EXISTS S; DROP TABLE IF EXISTS T;
%sql CREATE TABLE R (A int); CREATE TABLE S (A int); CREATE TABLE T (A int);
for i in range(1,6):
    %sql INSERT INTO R VALUES (:i)
for i in range(1,11,3):
    %sql INSERT INTO T VALUES (:i)

Done.
Done.
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


Let's try to get $R \cap (S \cup T) = \{1,4\}$

In [9]:
%%sql SELECT DISTINCT R.A FROM R, S, T
WHERE R.A=S.A OR R.A=T.A

Done.


[]

**Why is the return set empty?**

*Recall the order of operations for this query:*
1. We take the cross-product of `R,S,T`
2. We filter this intermediate table by the conditions of the `WHERE` clause.  

Let's see (1):

In [10]:
%sql SELECT DISTINCT R.A FROM R, S, T;

Done.


[]

The cross-product query is empty because `S` is empty!

Union
-----

Let's try a different way using a new SQL command, `UNION`:

In [11]:
%%sql
SELECT R.A FROM R, S WHERE R.A=S.A
UNION -- this is an explicit keyword!
SELECT R.A FROM R, T WHERE R.A=T.A

Done.


[(1,), (4,)]

* There are no duplicates (union is a set)
* If we wanted duplicates, we could use UNION ALL
  * R = {1,2,3,4,5}
  * S = {1,2,3,4,5}
  * T = {1,4,7,10}

In [12]:
%sql DROP TABLE IF EXISTS S; CREATE TABLE S (A int);
for i in range(1,6):
    %sql INSERT INTO S VALUES (:i)

Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


In [13]:
%%sql
-- UNION ALL example- notice that 1 and 4 occur twice!
SELECT R.A FROM R, S WHERE R.A=S.A
UNION ALL
SELECT R.A FROM R, T WHERE R.A=T.A

Done.


[(1,), (2,), (3,), (4,), (5,), (1,), (4,)]

Other Set Operations: INTERSECT, EXCEPT
-------------------------------------

Two examples below... see more in the next section

In [14]:
%%sql
SELECT R.A FROM R, S, T WHERE R.A = S.A
INTERSECT
SELECT R.A FROM R, S, T WHERE R.A = T.A

Done.


[(1,), (4,)]

In [15]:
%%sql
SELECT R.A FROM R, S, T WHERE R.A = S.A
EXCEPT
SELECT R.A FROM R, S, T WHERE R.A = T.A

Done.


[(2,), (3,), (5,)]

Running example for rest of this notebook: BAGELS
===========================

The selection of which had nothing to do with how hungry the person writing this section was.

Outline of topics:
* Nested queries: motivation & examples
* Aggregation: Overview, with group-by
* Null Values, Outer vs. Inner Joins

Story:
------
* *eBagel* is a ground-breaking new tech startup in the red-hot NoSQL-based bagel enterprise space
* eBagel has just received $100M in venture capital funding.  However, their sales numbers are dropping off, and you have been brought in to analyze the data and figure out what's going wrong

Let's begin by populating the following tables:
> Franchise(name TEXT, db_type TEXT)

> Store(franchise TEXT, location TEXT)

> Bagel(name TEXT, price MONEY, made_by TEXT)

> Purchase(bagel_name TEXT, franchise TEXT, date INT, quantity INT, purchaser_age INT)

Let's quickly load the data first

Union
-----

First, let's find franchises that have locations in PA **OR** NYC to see who the potential competition is:

In [16]:
%%sql 
SELECT franchise FROM store WHERE location = 'NYC'
UNION
SELECT franchise FROM store WHERE location = 'PA';

Done.


[(u'BAGEL CORP',), (u'Bobs Bagels',), (u'eBagel',)]

Intersect: Subtle problem...
--------------------------

eBagel's CEO is curious to know more about the back-end technologies of Bagel companies that have successfully spread across multiple locations.  Let's try to use an `INTERSECT` operator to find the database types of franchises which have stores in PA **AND** NYC:

In [17]:
%%sql
SELECT f.db_type
FROM franchise f, store s 
WHERE f.name = s.franchise AND s.location = 'NYC'
INTERSECT
SELECT f.db_type
FROM franchise f, store s 
WHERE f.name = s.franchise AND s.location = 'PA'

Done.


[(u'MySQL',), (u'NoSQL',)]

*What happened here?*

If we look at the data, we should only have gotten "MySQL" as a result:

In [18]:
%%sql 
SELECT f.name, s.location, f.db_type
FROM franchise f, store s 
WHERE f.name = s.franchise;

Done.


[(u'Bobs Bagels', u'NYC', u'NoSQL'),
 (u'eBagel', u'PA', u'NoSQL'),
 (u'BAGEL CORP', u'Chicago', u'MySQL'),
 (u'BAGEL CORP', u'NYC', u'MySQL'),
 (u'BAGEL CORP', u'PA', u'MySQL')]

So what happened?  We can see why this occurs by breaking the query operation down:

In [19]:
%%sql SELECT f.db_type 
FROM franchise f, store s 
WHERE f.name = s.franchise AND s.location = 'NYC'

Done.


[(u'NoSQL',), (u'MySQL',)]

In [20]:
%%sql SELECT f.db_type
FROM franchise f, store s
WHERE f.name = s.franchise AND s.location = 'PA'

Done.


[(u'NoSQL',), (u'MySQL',)]

Essentially, the problem is that we did the `INTERSECT` operation over the return attributes, not the attributes we actually wanted to use!

Nested queries
========

Instead, let's see one solution to the above problem that uses *nested queries*: i.e. subqueries that return relations which our main query then operates over:

In [21]:
%%sql
SELECT f.db_type
FROM franchise f
WHERE f.name IN (
    SELECT s.franchise FROM store s WHERE s.location = 'NYC')
  AND f.name IN (
    SELECT s.franchise FROM store s WHERE s.location = 'PA');

Done.


[(u'MySQL',)]

Another nested queries example: eBagel's CEO wants to know what databases support bagel companies that appeal to to the crucial twenty-something demographic:

In [22]:
%%sql
SELECT f.db_type
FROM franchise f
WHERE f.name IN (
    SELECT b.made_by
    FROM bagel b, purchase p
    WHERE b.name = p.bagel_name 
      AND p.purchaser_age >= 20 AND p.purchaser_age < 30);

Done.


[(u'NoSQL',), (u'MySQL',)]

Nested queries are powerful and convenient!  Could we accomplish the above without them however?  Would this return the same result?

In [23]:
%%sql
SELECT f.db_type
FROM franchise f, bagel b, purchase p
WHERE f.name = b.made_by 
  AND b.name = p.bagel_name 
  AND p.purchaser_age >= 20 AND p.purchaser_age < 30;

Done.


[(u'MySQL',), (u'NoSQL',), (u'MySQL',), (u'NoSQL',)]

**Beware of duplicates!**  We can add a `DISTINCT` however and they will be equivalent- try this for yourself.

We can also use the following over the results of nested subqueries:
* `ALL`
* `ANY`
* `EXISTS`

**Unfortunately, `ALL` and `ANY` are not supported in SQLite (the DBMS we're using here)- see the lecture notes for examples.**  However we can show an example of `EXISTS`.  Suppose we want to answer the following question: does eBagel have *any* products which are cheaper than *any* of its competitors'?

In [24]:
%%sql
SELECT b.name, b.price
FROM bagel b
WHERE b.made_by = 'eBagel'
  AND EXISTS (SELECT name FROM bagel WHERE made_by <> 'eBagel' AND price > b.price);

Done.


[(u'eBagel Expansion Pack', 1.99)]

Note that this is a *correlated query* because values from the main outer query are involved in the inner nested query.  Especially take notice of the variable scoping here.  Note also that this can be expressed as a single SFW query- can you figure out how?

Summary so far:
--------------

SQL:
* Provides a high-level declarative language for manipulating data (DML)
* The workhorse is the SFW block
* Set operators are powerful- but have some subtleties to be aware of!
* Powerful, nested queries are also allowed
    * **Note that this becomes especially helpful when using a more full-featured DBMS than SQLite!**

Aggregation
=======

SQL supports several aggregation operations:
* `SUM`
* `COUNT`
* `AVG`
* `MIN`
* `MAX`

*Except `COUNT`*, all aggregation operators apply to a single attribute only

Here are some simple examples:

In [25]:
%sql SELECT AVG(price) FROM bagel WHERE made_by = 'eBagel';

Done.


[(14.989999999999998,)]

Note that you can define the attribute labels of the output schema, which is convenient for aggregation operations:

In [26]:
%sql SELECT COUNT(*) AS "Number of Stores in PA" FROM store WHERE location = 'PA';

Done.


[(2,)]

What if we want to know how many total locations there are of relevance in the Bagel 2.0 space?

In [27]:
%sql SELECT COUNT(location) FROM store;

Done.


[(5,)]

What we actually wanted:

In [28]:
%sql SELECT COUNT(DISTINCT location) FROM store;

Done.


[(3,)]

Next, you've purchased some industry sales data, and want to get a sense of how big the bagel space is exactly.  How can we do this easily and efficiently, *all in SQL*?

In [29]:
%%sql
SELECT SUM(b.price * p.quantity) AS net_sales
FROM bagel b, purchase p
WHERE b.name = p.bagel_name;

Done.


[(432.99,)]

That's a huge market opportunity!  Now, eBagel's CEO is quite excited, and wants to see more detail, such as how the distribution across companies breaks down.  Luckily, **grouping and aggregation can be combined!**

In [30]:
%%sql
SELECT b.made_by, SUM(b.price * p.quantity) AS revenue
FROM bagel b, purchase p
WHERE b.made_by = p.franchise AND b.name = p.bagel_name
GROUP BY b.made_by;

Done.


[(u'BAGEL CORP', 12.87), (u'Bobs Bagels', 85.98), (u'eBagel', 272.63)]

These are exciting results!

Finally, to cap off the analysis, you want to see the distribution of revenue by product.  *However*, in order to paint a more effective high-level picture, you only want to see best-sellers, which you define as products with more that a dozen total sales.

To do this we introduce the `HAVING` clause, which conditions *on aggregates*

In [31]:
%%sql
SELECT b.name, SUM(b.price * p.quantity) AS sales
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING SUM(p.quantity) > 12;

Done.


[(u'Plain with shmear', 84.50999999999999), (u'eBagel Expansion Pack', 272.63)]

Let's break down this last query into its actual evaluation steps:

First, the FROM-WHERE segment of the clause is evaluated:

In [32]:
%%sql
SELECT *
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise;

Done.


[(u'Plain with shmear', 1.99, u'Bobs Bagels', u'Plain with shmear', u'Bobs Bagels', 1, 12, 28),
 (u'Plain with shmear', 1.99, u'Bobs Bagels', u'Plain with shmear', u'Bobs Bagels', 4, 24, None),
 (u'Egg with shmear', 2.39, u'Bobs Bagels', u'Egg with shmear', u'Bobs Bagels', 2, 6, 47),
 (u'eBagel Expansion Pack', 1.99, u'eBagel', u'eBagel Expansion Pack', u'eBagel', 1, 137, 5),
 (u'Plain with shmear', 0.99, u'BAGEL CORP', u'Plain with shmear', u'BAGEL CORP', 2, 12, 24),
 (u'Plain with shmear', 0.99, u'BAGEL CORP', u'Plain with shmear', u'BAGEL CORP', 3, 1, 17)]

(*Aside: what would we have gotten if we had left out `AND b.made_by = p.franchise` in the WHERE clause?  Breaking down a complex query into its component steps is the best way to debug things like this!*)

Next, the GROUP-BY is applied:

In [33]:
%%sql
SELECT 
    b.name,
    GROUP_CONCAT(b.price, ',') AS prices,
    GROUP_CONCAT(b.made_by, ',') AS made_bys,
    bagel_name, 
    GROUP_CONCAT(p.franchise, ',') AS franchises,
    GROUP_CONCAT(p.date, ',') AS dates,
    GROUP_CONCAT(p.quantity, ',') AS quantities,
    GROUP_CONCAT(p.purchaser_age, ',') AS purchaser_ages
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name;

Done.


[(u'Egg with shmear', u'2.39', u'Bobs Bagels', u'Egg with shmear', u'Bobs Bagels', u'2', u'6', u'47'),
 (u'Plain with shmear', u'1.99,1.99,0.99,0.99', u'Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP', u'Plain with shmear', u'Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP', u'1,4,2,3', u'12,24,12,1', u'28,24,17'),
 (u'eBagel Expansion Pack', u'1.99', u'eBagel', u'eBagel Expansion Pack', u'eBagel', u'1', u'137', u'5')]

*Note that the `GROUP_CONCAT` functions above are just for nicer display, as otherwise SQLite will choose a single arbitrary relation's values to display for the column in such a query*

Next, the `HAVING` condition is applied:

In [34]:
%%sql
SELECT 
    b.name,
    GROUP_CONCAT(b.price, ',') AS prices,
    GROUP_CONCAT(b.made_by, ',') AS made_bys,
    bagel_name, 
    GROUP_CONCAT(p.franchise, ',') AS franchises,
    GROUP_CONCAT(p.date, ',') AS dates,
    SUM(p.quantity) AS total_quantity,
    GROUP_CONCAT(p.purchaser_age, ',') AS purchaser_ages
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING SUM(p.quantity) > 12;

Done.


[(u'Plain with shmear', u'1.99,1.99,0.99,0.99', u'Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP', u'Plain with shmear', u'Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP', u'1,4,2,3', 49, u'28,24,17'),
 (u'eBagel Expansion Pack', u'1.99', u'eBagel', u'eBagel Expansion Pack', u'eBagel', u'1', 137, u'5')]

And finally, the aggregates and other operations in the select clause (projection) are applied:

In [35]:
%%sql
SELECT b.name, SUM(b.price * p.quantity) AS sales
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING SUM(p.quantity) > 12;

Done.


[(u'Plain with shmear', 84.50999999999999), (u'eBagel Expansion Pack', 272.63)]